In [1]:
#| default_exp select

In [2]:
#|export

import jax
from jax import lax
import jax.numpy as jnp
from typing import Tuple
from chewc.meiosis import create_gamete
from chewc.structs import GeneticMap, Population
from jax.tree_util import tree_map

from functools import partial
@partial(jax.jit, static_argnames=("k",))
def select_top_k(population: Population, values: jnp.ndarray, k: int) -> Population:
    """Select top-k individuals by `values` (1-D)."""
    vals = jnp.ravel(values).astype(jnp.float32)  # (n_ind,)
    _, top_idx = lax.top_k(vals, k)               # (k,)
    return tree_map(lambda x: x[top_idx], population)



def _solve_for_c_internal(lambda_val: float, g: jnp.ndarray, A: jnp.ndarray) -> jnp.ndarray:
    """Internal MME solver for OCS."""
    N = g.shape[0]
    top_left = 2 * lambda_val * A
    top_right = jnp.ones((N, 1))
    bottom_left = jnp.ones((1, N))
    bottom_right = jnp.zeros((1, 1))
    M_top = jnp.hstack([top_left, top_right])
    M_bottom = jnp.hstack([bottom_left, bottom_right])
    M = jnp.vstack([M_top, M_bottom])
    b = jnp.concatenate([g, jnp.array([1.0])])
    solution = jnp.linalg.solve(M, b)
    return solution[:-1]


@partial(jax.jit, static_argnames=('max_iterations',))
def find_optimal_contributions(
    g: jnp.ndarray,
    A: jnp.ndarray,
    delta_F: float,
    start_lambda: float = 1.0,
    tolerance: float = 1e-6,
    max_iterations: int = 100
) -> jnp.ndarray:
    """
    JIT-compiled function using jax.lax.while_loop to find optimal contributions.
    """
    initial_state = (0, start_lambda, jnp.inf)

    def condition_fn(state):
        i, _, error = state
        return jnp.logical_and(i < max_iterations, error > tolerance)

    def body_fn(state):
        i, lambda_val, _ = state
        c = _solve_for_c_internal(lambda_val, g, A)
        delta_F_current = 0.5 * c.T @ A @ c
        error = jnp.abs(delta_F_current - delta_F)
        new_lambda = lambda_val * (delta_F_current / delta_F)
        return (i + 1, new_lambda, error)

    _, final_lambda, _ = lax.while_loop(condition_fn, body_fn, initial_state)

    final_c = _solve_for_c_internal(final_lambda, g, A)
    return final_c


def select_with_ocs(
    population: Population,
    breeding_values: jnp.ndarray,
    delta_F: float,
    k: int
) -> Tuple[Population, jnp.ndarray]:
    """
    Selects top-k individuals and calculates their optimal contributions.
    """
    # 1. Pre-select top-k candidates to reduce matrix sizes
    candidates = select_top_k(population, breeding_values, k=k)
    candidate_bvs = select_top_k(breeding_values, breeding_values, k=k)

    # 2. Build the full A matrix for the candidates
    # NOTE: This is a simplified approach. For larger `k`, a more
    # efficient method to build A would be needed.
    remapped_pedigree = remap_pedigree(candidates.meta)
    A_inv_sparse = build_a_inverse_sparse(remapped_pedigree)
    A = jnp.linalg.inv(A_inv_sparse.todense())

    # 3. Find optimal contributions
    c = find_optimal_contributions(candidate_bvs, A, delta_F)
    
    # 4. Normalize contributions to be non-negative and sum to 1
    contributions = jnp.maximum(0, c)
    contributions /= jnp.sum(contributions)

    return candidates, contributions


/home/glect/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
#| hide
import nbdev; nbdev.nbdev_export()